In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
import itertools
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
pd.set_option('display.max_Columns', 100)

In [2]:
train = pd.read_csv('../inputs/train.csv')
print(train.shape)
test = pd.read_csv('../inputs/test.csv')
print(test.shape)

(8359, 16)
(8360, 11)


In [3]:
def creansing(df_input):
    df_input['User_Score_tbd'] = (df_input['User_Score'] == 'tbd').astype(int)
    df_input['User_Score'] = df_input['User_Score'].replace({'nan': np.NaN, 'tbd': np.NaN}).astype(float)
    df_input['User_Score_int'] = df_input['User_Score'].round()
    df_input['Critic_Score_int'] = df_input['Critic_Score'].round(-1)/10
    df_input['User_Count_log'] = df_input['User_Count'].apply(np.log1p)
    df_input['Year'] = df_input['Year_of_Release'].astype(float)
    df_input.drop(columns=['Year_of_Release'], inplace=True)
    return df_input

In [4]:
def fill_null_year(train, test):
    whole = pd.concat([train, test])
    year = whole.groupby(['Publisher', 'Platform'])['Year'].median().reset_index()
    year = year.fillna(0).rename(columns={'Year': 'MedYear'})
    train = pd.merge(train, year, on=['Publisher', 'Platform'], how='left')
    test = pd.merge(test, year, on=['Publisher', 'Platform'], how='left')
    train.loc[train['Year'].isnull(), 'Year'] = train['MedYear']
    test.loc[test['Year'].isnull(), 'Year'] = test['MedYear']
    return train.drop(columns='MedYear'), test.drop(columns='MedYear')

In [5]:
def fill_null_developer(train, test):
    whole = pd.concat([train, test])
    
    # 'Genre', 'Platform'で最大数の'Developer'で埋める
    mode_dev = whole.groupby(['Genre', 'Platform'])[['Developer']].apply(lambda x: x.mode()).reset_index()
    mode_dev = mode_dev.drop_duplicates(subset=['Genre', 'Platform'])
    mode_dev = mode_dev.rename(columns={'Developer':'devlop'})
    train = pd.merge(train, mode_dev, on=['Genre', 'Platform'], how='left')
    test = pd.merge(test, mode_dev, on=['Genre', 'Platform'], how='left')
    train.loc[train['Developer'].isnull(), 'Developer'] = train['devlop']
    test.loc[test['Developer'].isnull(), 'Developer'] = test['devlop']

    return train.drop(columns='devlop'), test.drop(columns='devlop')

In [6]:
train = creansing(train)
test = creansing(test)
train, test = fill_null_year(train, test)
# train, test = fill_null_developer(train, test)

In [7]:
n_fold = 5

### label_encode

In [8]:
def get_non_overlapping(train: pd.DataFrame, test: pd.DataFrame, column: str):
    """train/testにしか出てこない値を調べる"""
    only_in_train = set(train[column].unique()) - set(test[column].unique())
    only_in_test = set(test[column].unique()) - set(train[column].unique())
    non_overlapping = only_in_train.union(only_in_test)
    return non_overlapping

def category2num(train: pd.DataFrame, test: pd.DataFrame, columns: list):
    train_ = train.copy()
    test_ = test.copy()
    for column in columns:
        non_overlapping = get_non_overlapping(train, test, column)
        try:
            if train[column].dtype == np.dtype("O"):
                # dtypeがobjectなら欠損は'missing' クラスにする
                train_[column] = train[column].fillna("missing")
                test_[column] = test[column].fillna("missing")
                train_[column] = train_[column].map(lambda x: x if x not in non_overlapping else "other")
                test_[column] = test_[column].map(lambda x: x if x not in non_overlapping else "other")
            else:
                # dtypeがint/floatなら欠損は'-1'とする
                train_[column] = train[column].fillna(-1)
                test_[column] = test[column].fillna(-1)
                train_[column] = train_[column].map(lambda x: x if x not in non_overlapping else -2)
                test_[column] = test_[column].map(lambda x: x if x not in non_overlapping else -2)

            le = preprocessing.LabelEncoder()
            concatenated = pd.concat([train_, test_], axis=0).reset_index(drop=True)
            le.fit(concatenated[column])
            train_[column] = le.transform(train_[column])
            test_[column] = le.transform(test_[column])
        except Exception:
            import pdb
            pdb.set_trace()
    return train_, test_

In [9]:
def create_label_encoding(df_train, df_test):
    tr, te = category2num(df_train, df_test, ['Platform', 'Genre', 'Developer', 'Rating'])
    return tr, te

### count_encode

In [10]:
def count_encoding(df_train, df_test, col):
    tr = df_train[col]
    te = df_test[col]
    whole = pd.concat([tr, te])
    vc = whole.value_counts(dropna=False)
    tr_ = pd.DataFrame(tr.map(vc)).add_prefix('CE_')
    te_ = pd.DataFrame(te.map(vc)).add_prefix('CE_')
    return tr_, te_

def create_count_encoding_feature(df_train, df_test):
    tr = pd.DataFrame()
    te = pd.DataFrame()
    for col in ['Platform', 'Genre', 'Publisher', 'Developer', 'Rating', 'User_Score_int', 'Critic_Score_int']:
        tr_, te_ = count_encoding(df_train, df_test, col)
        tr = pd.concat([tr, tr_], axis=1)
        te = pd.concat([te, te_], axis=1)
    return tr, te

### target_encode

In [11]:
def target_encoding(src, dst, group_col, target_col, aggfunc):
    agg_dict = {}
    for tc in target_col:
        agg_dict[tc] = aggfunc
    target = src.groupby(group_col).agg(agg_dict).reset_index()
    
    merge = pd.merge(dst[group_col], target, on=group_col, how='left').set_index(dst.index)
    suffix = '_'.join(group_col)
    return merge.drop(columns=group_col).add_prefix('TGE_').add_suffix('_{}_by_{}'.format(aggfunc, suffix))

In [12]:
def crete_target_enc_train_test(df_train, df_test, group_col, target_col, aggfunc):
    target = pd.DataFrame()
    kf = KFold(n_splits=n_fold, shuffle=True, random_state=0)
    for train_index, eval_index in kf.split(df_train):
        kf_train = df_train.iloc[train_index]
        kf_eval = df_train.iloc[eval_index]
        tmp = target_encoding(kf_train, kf_eval, group_col, target_col, aggfunc)
        target = pd.concat([target, tmp])
    # train
    tr = target.sort_index()
    # test
    te = target_encoding(df_train, df_test, group_col, target_col, aggfunc)
    return tr, te

In [13]:
def create_target_encoding_sub(df_train, df_test, tr, te, group, target, aggfunc):
    tr_, te_ = crete_target_enc_train_test(df_train, df_test, group, target, aggfunc)
    tr = pd.concat([tr, tr_], axis=1)
    te = pd.concat([te, te_], axis=1)
    return tr, te

In [14]:
def create_target_encoding(df_train, df_test):
    tr = pd.DataFrame()
    te = pd.DataFrame()
#     target = ['Global_Sales', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
    target = ['Global_Sales']
    groups = [
        ['Platform'],
        ['Genre'],
        ['Rating'],
        ['User_Score_int'],
        ['Critic_Score_int'],
        
        ['Platform', 'Genre'],
        ['Platform', 'Rating'],
        ['Genre', 'Rating'],
        
        ['User_Score_int', 'Platform'],
        ['User_Score_int', 'Genre'],
        ['User_Score_int', 'Rating'],
        ['Critic_Score_int', 'Platform'],
        ['Critic_Score_int', 'Genre'],
        ['Critic_Score_int', 'Rating'],
        ['Critic_Score_int', 'User_Score_int'],
    ]
    for gr in groups:
        tr, te = create_target_encoding_sub(df_train, df_test, tr, te, gr, target, 'mean')
    for gr in groups:
        tr, te = create_target_encoding_sub(df_train, df_test, tr, te, gr, target, 'std')
    
    return tr, te

### word (TruncatedSVD)

In [15]:
def word_pivot(namelist, wordlist):
    pivot = pd.DataFrame(namelist)
    for word in wordlist:
        pivot[word] = pivot['Name'].str.contains(word)
    return pivot

In [16]:
def create_wordlist(train, test):
    trainName = train['Name'].str.lower()
    testName = test['Name'].str.lower()
    trainName = trainName.str.replace('[^A-Za-z]', ' ')
    testName = testName.str.replace('[^A-Za-z]', ' ')
    trainlen = len(trainName)
    print('trainlen', trainlen)
    name = pd.concat([trainName, testName]).reset_index(drop=True)
    print('name', len(name))

    wordlist = []
    for n in list(name):
        wordlist = wordlist + (str(n).split())
    print('wordlist', len(wordlist))
    
    df_name = pd.DataFrame(pd.Series(wordlist, name='name_count').value_counts()).reset_index()
    df_name.rename(columns={'index': 'name'}, inplace=True)
    print('df_name', len(df_name))

    # 4文字以上、10回以上出現している単語
    df_name['word_len'] = df_name['name'].str.len()
    df_longname = df_name[df_name['word_len'] >= 4]
    df_longname = df_longname[df_longname['name_count'] >= 10]
    print('df_longname', len(df_longname))
    
    # 含まれるかどうか
    train_pivot = word_pivot(trainName, list(df_longname['name']))
    print('train_pivot', train_pivot.shape)
    test_pivot = word_pivot(testName, list(df_longname['name']))
    print('test_pivot', test_pivot.shape)
    
    # trainとtestの両方に存在するものだけ(train>3, test>0)
    train_sum = pd.DataFrame(train_pivot.drop(columns=['Name']).sum(), columns=['tr_count'])
    test_sum = pd.DataFrame(test_pivot.drop(columns=['Name']).sum(), columns=['te_count'])
    train_test_sum = pd.concat([train_sum, test_sum], axis=1).reset_index()
    new_wordlist = train_test_sum[(train_test_sum['tr_count']>3)&(train_test_sum['te_count']>0)].reset_index(drop=True)
    
    train_pivot2 = word_pivot(trainName, list(new_wordlist['index']))
    print('train_pivot2', train_pivot2.shape)
    test_pivot2 = word_pivot(testName, list(new_wordlist['index']))
    print('test_pivot2', test_pivot2.shape)
    
    train_pivot2.to_feather('../inputs/train_word.f')
    test_pivot2.to_feather('../inputs/test_word.f')
    
    return train_pivot2, test_pivot2

In [17]:
skip_create_wordlist = True

In [18]:
def create_word_features(df_train, df_test):
    
    if skip_create_wordlist:
        train_word = pd.read_feather('../inputs/train_word.f')
        test_word = pd.read_feather('../inputs/test_word.f')
    else:
        train_word, test_word = create_wordlist(df_train, df_test)
        
    wd_tr = train_word.drop(columns=['Name']).fillna(0).astype(int).add_prefix('WD_')
    wd_te = test_word.drop(columns=['Name']).fillna(0).astype(int).add_prefix('WD_')
    print('wd_tr', wd_tr.shape)
    print('wd_te', wd_te.shape)
    
    return wd_tr, wd_te

#     ntopic = 10
#     lda = LatentDirichletAllocation(n_components=ntopic)
#     x_tr = pd.DataFrame(lda.fit_transform(wd_tr)).add_prefix('LDA_')
#     x_te = pd.DataFrame(lda.fit_transform(wd_te)).add_prefix('LDA_')
    
#     svd = TruncatedSVD(ntopic)
#     x_tr = pd.DataFrame(svd.fit_transform(wd_tr)).add_prefix('SVD_')
#     x_te = pd.DataFrame(svd.fit_transform(wd_te)).add_prefix('SVD_')
#     return x_tr, x_te
    
#     tr_ = pd.concat([wd_tr, x_tr], axis=1)
#     te_ = pd.concat([wd_te, x_te], axis=1)
#     return tr_, te_

### LDA

In [19]:
def LDA_topic(df_input, ntopic, index, column):

    df_input['dummy'] = 1
    pvt_tbl = pd.pivot_table(df_input, index=index, columns=column, values='dummy', aggfunc='count')
    pvt_tbl.fillna(0, inplace=True)
    
    prefix = 'LDA_{}_'.format(column)
    suffix = '_by_{}'.format(index)
    
    lda = LatentDirichletAllocation(n_components=ntopic)
    lda_out = pd.DataFrame(lda.fit_transform(pvt_tbl), index=pvt_tbl.index).add_prefix(prefix).add_suffix(suffix)
    return lda_out.reset_index()

In [20]:
def create_lda_features(df_train, df_test):
    cat_list = ['Platform', 'Genre', 'Developer', 'Rating', 'Publisher']
    tr = df_train[cat_list].copy()
    te = df_test[cat_list].copy()
    whole = pd.concat([tr, te])
    
    for a, b in itertools.permutations(cat_list, 2):
        lda_o = LDA_topic(whole, 5, index=a, column=b)
        tr = pd.merge(tr, lda_o, on=a, how='left')
        te = pd.merge(te, lda_o, on=a, how='left')
        
    return tr.drop(columns=cat_list), te.drop(columns=cat_list)

### salesの割合

In [21]:
sales_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'] #, 'Global_Sales']

def sales_groupby(df_train, df_test, groupkey):
    tr = df_train[groupkey].copy()
    te = df_test[groupkey].copy()
    grp_sum = df_train[sales_columns].groupby(df_train[groupkey]).sum()
    
    # 列方向の割合
#     salescol_sum = grp_sum.sum()
#     for c in sales_columns:
#         grp_sum['Rate1_'+c] = grp_sum[c] / salescol_sum[c]
#     grp_sum.drop(columns=['Rate1_Global_Sales'], inplace=True)
    
    for c in sales_columns:
        grp_sum['Rank_'+c] = grp_sum[c].rank()

    grp_sum.drop(columns=sales_columns, inplace=True)
    grp_sum = grp_sum.add_suffix('_by_{}'.format(groupkey))

    tr = pd.merge(tr, grp_sum.reset_index(), on=groupkey, how='left').drop(columns=groupkey)
    te = pd.merge(te, grp_sum.reset_index(), on=groupkey, how='left').drop(columns=groupkey)
    return tr, te

In [22]:
def create_sales_feature(df_train, df_test):
    tr = pd.DataFrame(index=df_train.index)
    te = pd.DataFrame(index=df_test.index)
    keys = ['Platform', 'Genre', 'Developer', 'Rating', 'Year']
    for k in keys:
        tr_, te_ = sales_groupby(train, test, k)
        tr = pd.concat([tr, tr_], axis=1)
        te = pd.concat([te, te_], axis=1)
    return tr, te

### Publisher/Developer (LDA)

In [23]:
def create_categorical_LDA_feature(df_train, df_test, category):
    df_whole = pd.concat([df_train, df_test])
    plat_pivot = df_whole.pivot_table(index=category, columns='Platform', values='Name', aggfunc='count')
    genre_pivot = df_whole.pivot_table(index=category, columns='Genre', values='Name', aggfunc='count')
    year_pivot = df_whole.pivot_table(index=category, columns='Year', values='Name', aggfunc='count')
    print(category, 'plat_pivot', len(plat_pivot))
    print(category, 'genre_pivot', len(genre_pivot))
    print(category, 'year_pivot', len(year_pivot))
    
    all_pivot = plat_pivot.join(genre_pivot, how='outer')
    all_pivot = all_pivot.join(year_pivot, how='outer')
#     all_pivot = pd.concat([plat_pivot, genre_pivot], axis=1)
#     all_pivot = pd.concat([all_pivot, year_pivot], axis=1)
    all_pivot.fillna(0, inplace=True)
    print(category, 'all_pivot', len(all_pivot))
    
    prefix = 'LDA_{}_'.format(category)
    lda = LatentDirichletAllocation(n_components=10)
    lda_out = pd.DataFrame(lda.fit_transform(all_pivot), index=all_pivot.index).add_prefix(prefix).reset_index()
    
    tr = pd.merge(df_train[[category]], lda_out, on=category, how='left').drop(columns=[category])
    te = pd.merge(df_test[[category]], lda_out, on=category, how='left').drop(columns=[category])
    return tr, te

In [24]:
def create_publisher_feature(df_train, df_test):
    return create_categorical_LDA_feature(df_train, df_test, 'Publisher')

In [25]:
def create_developer_feature(df_train, df_test):
    return create_categorical_LDA_feature(df_train, df_test, 'Developer')

### 特徴量生成

In [26]:
processors = [
#     create_sales_feature,
    create_count_encoding_feature,
    create_target_encoding,
    create_lda_features,
    create_word_features,
    create_publisher_feature,
#     create_developer_feature,
]

In [27]:
def create_feature(train, test):
    train_cp = train.copy()
    test_cp = test.copy()
    
    for func in processors:
        tr, te = func(train_cp, test_cp)
        train_cp = pd.concat([train_cp, tr], axis=1)
        test_cp = pd.concat([test_cp, te], axis=1)

    # label enc
#     train_cp, test_cp = create_label_encoding(train_cp, test_cp)
    
    print('- train', train_cp.shape)
    print('- test', test_cp.shape)
    assert len(train) == len(train_cp)
    assert len(test) == len(test_cp)
    return train_cp, test_cp

In [28]:
train_, test_ = create_feature(train, test)

wd_tr (8359, 815)
wd_te (8360, 815)
Publisher plat_pivot 581
Publisher genre_pivot 581
Publisher year_pivot 581
Publisher all_pivot 581
- train (8359, 982)
- test (8360, 977)


In [29]:
train_.to_feather('../inputs/train_cat_feature.f')
test_.to_feather('../inputs/test_cat_feature.f')

In [30]:
# label enc
train_, test_ = create_label_encoding(train_, test_)
train_.to_feather('../inputs/train_feature.f')
test_.to_feather('../inputs/test_feature.f')

assert len(train) == len(train_)
assert len(test) == len(test_)

#### 30
- NameLDAやめる
- train (8359, 982)
- test (8360, 977)

#### 29
- Nameを減らして、LDA(10)追加
- train (8359, 992)
- test (8360, 987)

#### 27
- YearのNaNを埋める/26やめる
- train (8359, 1438)
- test (8360, 1433)

#### 26
- DeveloperごとのcountをLDA
- train (8359, 1449)
- test (8360, 1444)

#### 25
- Year, DeveloperのNaNを埋める
- train (8359, 1439)
- test (8360, 1434)

#### 23
- SalesのRank
- train (8359, 1459)
- test (8360, 1454)

#### 22
- PublisherごとのcountをLDA


In [31]:
train_.head()

,Name,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,User_Score_tbd,User_Score_int,Critic_Score_int,User_Count_log,Year,CE_Platform,CE_Genre,CE_Publisher,CE_Developer,CE_Rating,CE_User_Score_int,CE_Critic_Score_int,TGE_Global_Sales_mean_by_Platform,TGE_Global_Sales_mean_by_Genre,TGE_Global_Sales_mean_by_Rating,TGE_Global_Sales_mean_by_User_Score_int,TGE_Global_Sales_mean_by_Critic_Score_int,TGE_Global_Sales_mean_by_Platform_Genre,TGE_Global_Sales_mean_by_Platform_Rating,TGE_Global_Sales_mean_by_Genre_Rating,TGE_Global_Sales_mean_by_User_Score_int_Platform,TGE_Global_Sales_mean_by_User_Score_int_Genre,TGE_Global_Sales_mean_by_User_Score_int_Rating,TGE_Global_Sales_mean_by_Critic_Score_int_Platform,TGE_Global_Sales_mean_by_Critic_Score_int_Genre,TGE_Global_Sales_mean_by_Critic_Score_int_Rating,TGE_Global_Sales_mean_by_Critic_Score_int_User_Score_int,TGE_Global_Sales_std_by_Platform,TGE_Global_Sales_std_by_Genre,TGE_Global_Sales_std_by_Rating,TGE_Global_Sales_std_by_User_Score_int,TGE_Global_Sales_std_by_Critic_Score_int,TGE_Global_Sales_std_by_Platform_Genre,TGE_Global_Sales_std_by_Platform_Rating,TGE_Global_Sales_std_by_Genre_Rating,...,WD_surfer,WD_faction,WD_gears,WD_slug,WD_tenchu,WD_dual,WD_joker,WD_others,WD_rivals,WD_professor,WD_crimson,WD_godzilla,WD_skies,WD_hakusho,WD_prophecy,WD_wings,WD_champion,WD_codename,WD_medieval,WD_town,WD_century,WD_atlantis,WD_scene,WD_rome,WD_dino,WD_groove,WD_volleyball,WD_wario,WD_machi,WD_mirai,WD_studio,WD_bandicoot,WD_director,WD_wall,WD_yoru,WD_kane,WD_west,WD_unreal,WD_turismo,WD_redemption,LDA_Publisher_0,LDA_Publisher_1,LDA_Publisher_2,LDA_Publisher_3,LDA_Publisher_4,LDA_Publisher_5,LDA_Publisher_6,LDA_Publisher_7,LDA_Publisher_8,LDA_Publisher_9
0,LEGO Batman: The Videogame,23,0,Warner Bros. Interactive Entertainment,180,97,0,28,306,74.0,17.0,7.9,22.0,425,1,0,8.0,7.0,3.135494,2010.0,1320,3370,235,77,1420,2830,2019,93.604962,61.883755,49.257785,106.536082,49.231678,59.678899,52.633663,56.848214,289.223684,112.428571,64.319444,81.140351,48.326531,50.646154,56.630137,444.419232,148.275892,72.187084,341.484617,73.847985,96.258706,76.873366,69.018125,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000142,0.000142,0.807465,0.000142,0.000142,0.031338,0.160205,0.000142,0.000142,0.000142
1,LEGO Indiana Jones: The Original Adventures,23,0,LucasArts,151,61,0,21,234,78.0,22.0,6.6,28.0,425,1,0,7.0,8.0,3.367296,2008.0,1320,3370,90,77,1420,1634,2366,96.597250,58.394680,49.148789,66.403374,101.466970,54.090000,48.867470,52.439815,213.160000,54.335079,58.293233,483.589744,89.802260,75.231343,96.758065,462.863031,120.188797,75.206074,174.818593,341.906251,91.001509,72.338596,62.128944,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.039358,0.157024,0.129704,0.000369,0.000369,0.147072,0.075802,0.000369,0.413444,0.036489
2,LEGO Batman: The Videogame,17,0,Warner Bros. Interactive Entertainment,56,44,0,27,128,73.0,5.0,7.4,10.0,425,1,0,7.0,7.0,2.397895,2010.0,1209,3370,235,77,1420,1634,2019,29.606403,59.681720,53.568595,64.981846,46.931655,48.185714,53.033898,58.454936,31.886792,58.615789,60.314286,45.701493,47.107692,53.884615,57.709821,69.859005,132.194405,81.714638,150.222097,77.663440,114.136366,82.460642,74.957223,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000142,0.000142,0.807465,0.000142,0.000142,0.031338,0.160205,0.000142,0.000142,0.000142
3,Combat,0,0,Atari,117,7,0,1,125,NaN,NaN,NaN,NaN,467,6,0,NaN,NaN,NaN,1982.0,133,3370,367,6623,6769,9129,8582,65.943662,62.435252,NaN,NaN,NaN,52.269231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.197493,149.649097,NaN,NaN,NaN,49.684249,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.019402,0.159094,0.000091,0.000091,0.000091,0.046073,0.011695,0.073007,0.690365,0.000091
4,LEGO Harry Potter: Years 5-7,23,0,Warner 